In [5]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
%matplotlib inline
import warnings;warnings.filterwarnings('ignore')
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats as spstats
from sklearn import metrics
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin, clone
from sklearn.preprocessing import Imputer, LabelEncoder, PolynomialFeatures
from sklearn.model_selection import KFold, StratifiedKFold, RandomizedSearchCV
from sklearn.externals import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [2]:
target = '标签'
uid = '申请编号'
uid_h = '历史申请编号'

In [ ]:
############## D3

d3= pd.read_csv('./data/train_History_Application.csv', header=0)
d3.shape
d3.head()

d4= pd.read_csv('./data/train_History_Payment.csv', header=0)
d4.shape
d4.head()

In [ ]:
####### D1 ~ D4 Exploration

# D3
d3_info = {}

d3_info['desc'] = d3.drop([uid, uid_h], axis=1).describe().T
d3_info['desc'].to_csv(f'./ana/0_cols_desc3.csv')

# D4
d4_info = {}

d4_info['desc'] = d4.drop([uid, uid_h], axis=1).describe().T
d4_info['desc'].to_csv(f'./ana/0_cols_desc4.csv')

In [ ]:
d3['合同状态'].value_counts()

In [ ]:
d4.loc[d4[uid]==173562, :].T.to_csv(f'./ana/9_t4_case1.csv')

In [ ]:
################  Load Features

''' Load '''
X = pd.read_csv('./tmp/train_d1234.csv', header=0, index_col=0)
X.shape

y = pd.read_csv('./data/train_label.csv', header=0, index_col=0)
y.shape

''' Merge '''
# Merge
xy = pd.merge(X, y, on=uid, how='inner')
xy.drop(uid, axis=1, inplace=True)
xy.shape

''' Split '''
# X, y
X = xy
y = X.pop(target)
X.shape
y.shape

In [3]:
# For kfold
n_folds = 5

# RF
param_fixed_rf = {
    'n_jobs' : 1,
    'oob_score' : True,
    'random_state':123,
    'verbose':0
}

# XGB
param_fixed_xgb = {
    'n_jobs' : 1,
    'eval_metric': 'auc',
    'seed' : 123,
    'silent' : 1,
    'verbose_eval':0
}

# LGB
param_fixed_lgb = {
    'n_jobs' : 1,
    'metric' : 'auc',
    'bagging_seed':123,
    'feature_fraction_seed':123,
    'verbose_eval' : 0
}

In [ ]:
############# boxcox

# feat = '商品价格'
# feat = '贷款申请前1年内征信查询次数'
feat = '出生日期距申请日期天数'

X[feat].describe()

In [ ]:
''' ori '''
sns.distplot(X[feat] + 1)
plt.show()

''' boxcox '''
inp = X[feat].apply('abs') + 1
l, opt_lmbda = spstats.boxcox(inp)
opt_lmbda
lmbda = 0
if opt_lmbda > 0:
    lmbda = opt_lmbda
sns.distplot(spstats.boxcox(inp, lmbda=lmbda))
plt.show()

In [ ]:
############## Fill Null

# Fill Null
imp =Imputer(missing_values=np.NaN, strategy="mean", axis=0)
data = imp.fit_transform(data_ori)
len(data)
len(data[0])

In [ ]:
############## X, y

''' y '''
y = data[:, 1]
len(y)
y[:10]

''' X '''
X = data[:, 2:]
len(X)
len(X[0])
X[0][:10]

pd.DataFrame(y).to_csv('./tmp/1_y.csv')
pd.DataFrame(X).to_csv('./tmp/1_X.csv')

In [ ]:
############## Check X, y

X1 = pd.read_csv('./tmp/1_X.csv', header=0, index_col=0)
X1.shape
X1.head()

y1 = pd.read_csv('./tmp/1_y.csv', header=0, index_col=0)
y1.shape
y1.head()

''' y '''
len(y)
y2 = y1['0'].values
len(y2)
(y == y2).sum()

''' X '''
len(X)
len(X[0])
X2 = X1.values
len(X2)
len(X2[0])
(X == X2).sum(axis=1)
140000 * 329

In [ ]:
n=0
y[n]
y2[n]

In [ ]:
f = '信用额度'
d1234[f].describe()
lmbda[f]

sns.distplot(d1234[f])

In [ ]:
### BoxCox

def get_boxcox(x):
    inp = x.apply('abs') + 1
    l, opt_lmbda = spstats.boxcox(inp)
    opt_lmbda
    lmbda = 0
    if opt_lmbda > 0:
        lmbda = opt_lmbda
    ret = spstats.boxcox(inp, lmbda=lmbda)
    return lmbda, ret

# Features
large_feats_drop = ('申请编号', '客户收入', '身份认证日期距申请日期天数')
large_feats_max = set(d1234.max()[d1234.max()>100].index)
large_feats_min = set(d1234.min()[d1234.min()<-100].index)
len(large_feats_max)
len(large_feats_min)
large_feats = large_feats_max.union(large_feats_min).difference(large_feats_drop)
len(large_feats)

lmbda = {}
for f in large_feats:
    lmbda[f], d1234[f] = get_boxcox(d1234[f])
    
#     f
#     sns.distplot(d1234[f])
#     plt.show()
#     sns.distplot(a)
#     plt.show()

In [ ]:
#### Polynomial

''' Feats  Poly '''
feats = np.load('./tmp/0_feats_poly.npy', allow_pickle=True).item()
len(feats)

# Poly
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
feats_poly = pd.DataFrame(poly.fit_transform(d1234[feats]), columns=poly.get_feature_names()).iloc[:, len(feats):]
feats_poly.shape

''' BoxCox '''
# BoxCox
large_feats_max = set(feats_poly.max()[feats_poly.max()>100].index)
large_feats_min = set(feats_poly.min()[feats_poly.min()<-100].index)
len(large_feats_max)
len(large_feats_min)
large_feats = large_feats_max.union(large_feats_min).difference(large_feats_drop)
len(large_feats)

lmbda = {}
for f in large_feats:
    lmbda[f], feats_poly[f] = get_boxcox(feats_poly[f])

''' Concat '''
pd.concat([d1234, feats_poly], axis=1)